Homework 1

In [2]:
import pandas as pd

sales_df = pd.read_csv(r"C:\Users\Asus\Downloads\sales_data.csv")

category_stats = sales_df.groupby('Category').agg({
    'Quantity': ['sum', 'max'],
    'Price': 'mean'
})

category_stats

Quantity           Price
                 sum max        mean
Category                            
Clothing         157  15   31.176471
Electronics      183  15  276.764706
Home             144  14   55.000000

In [3]:
top_products = sales_df.groupby(['Category', 'Product'])['Quantity'].sum()
top_products = top_products.groupby(level=0, group_keys=False).nlargest(1)
top_products

Category     Product        
Clothing     Jeans              15
Electronics  Smart TV           15
Home         Pressure Cooker    14
Name: Quantity, dtype: int64

In [4]:
sales_df['Total Sales'] = sales_df['Quantity'] * sales_df['Price']
date_max_sale = sales_df.groupby('Date')['Total Sales'].sum().idxmax()
date_max_sale

'2023-01-07'

In [8]:
customer_orders_df = pd.read_csv(r"C:\Users\Asus\Downloads\customer_orders.csv")

orders_count = customer_orders_df.groupby('CustomerID')['OrderID'].count()
customer_20plus = orders_count[orders_count>=20]
print(customer_20plus)

CustomerID
101    21
102    21
103    20
104    20
Name: OrderID, dtype: int64


In [11]:
avg_price_per_customer = customer_orders_df.groupby('CustomerID')['Price'].mean()
expensive_customer = avg_price_per_customer[avg_price_per_customer>120]
expensive_customer

CustomerID
102    138.095238
104    169.750000
Name: Price, dtype: float64

In [14]:
product_stats = customer_orders_df.groupby('Product').agg({
    'Quantity':'sum',
    'Price': lambda x: (x*customer_orders_df.loc[x.index, 'Quantity']).sum()
})
product_stats_filtered = product_stats[product_stats['Quantity']>=5]
product_stats_filtered

,Quantity,Price
Product,,
Cargo Pants,6,180
Dress Shirt,5,125
Formal Shirt,6,210
Smartphone,5,2000
Sport Shoes,5,200
Sunglasses,5,75
Wireless Earbuds,6,720


In [16]:
import sqlite3 as sql

conn = sql.connect(r"C:\Users\Asus\Downloads\population.db")
population_df = pd.read_sql("SELECT * from population",conn)
conn.close()

population_df.head()


,id,first_name,last_name,email,gender,salary,state
0,1,Armin,Coltart,acoltart0@abc.net.au,Male,368693,District of Columbia
1,2,Mia,Tuddenham,mtuddenham1@addthis.com,Female,154398,Florida
2,3,Kirsteni,Brafield,kbrafield2@arizona.edu,Female,1230304,Georgia
3,4,Phylis,Furlong,pfurlong3@merriam-webster.com,Female,1567795,California
4,5,Wandis,Loveredge,wloveredge4@hatena.ne.jp,Female,1136950,Alabama


In [18]:
salary_band = pd.read_excel(r"C:\Users\Asus\Downloads\population_salary_analysis.xlsx")
salary_band.head()

,Salary Band,Percentage,Average Salary,Median Salary,Number of population
0,"till $200,000",NaN,NaN,NaN,NaN
1,"$200,001 - $400,000",NaN,NaN,NaN,NaN
2,"$400,001 - $600,000",NaN,NaN,NaN,NaN
3,"$600,001 - $800,000",NaN,NaN,NaN,NaN
4,"$800,001 - $1,000,000",NaN,NaN,NaN,NaN


In [24]:
import pandas as pd

# Example salaries
population_df = pd.DataFrame({'Salary':[25000, 45000, 75000]})

# Define bands
bins = [0, 30000, 60000, 100000]  # lower and upper bounds
labels = ['Low', 'Medium', 'High']

# Assign SalaryBand
population_df['SalaryBand'] = pd.cut(population_df['Salary'], bins=bins, labels=labels)
print(population_df)



   Salary SalaryBand
0   25000        Low
1   45000     Medium
2   75000       High


In [25]:
band_stats = population_df.groupby('SalaryBand').agg(
    PopulationCount=('Salary', 'count'),
    AvgSalary=('Salary', 'mean'),
    MedianSalary=('Salary', 'median')
)

band_stats['Percentage'] = (band_stats['PopulationCount'] / population_df.shape[0]) * 100

print(band_stats)


            PopulationCount  AvgSalary  MedianSalary  Percentage
SalaryBand                                                      
Low                       1    25000.0       25000.0   33.333333
Medium                    1    45000.0       45000.0   33.333333
High                      1    75000.0       75000.0   33.333333


C:\Users\Asus\AppData\Local\Temp\ipykernel_10688\2814511069.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  band_stats = population_df.groupby('SalaryBand').agg(


In [28]:
state_stats = population_df.groupby(['State','SalaryBand']).agg(
    PopulationCount=('Salary','count'),
    AvgSalary=('Salary','mean'),
    MedianSalary=('Salary','median')
)

state_stats['Percentage'] = state_stats.groupby(level=0)['PopulationCount'].apply(lambda x: x / x.sum() * 100)
print(state_stats)


KeyError: 'State'